In [3]:
%load_ext autoreload
%autoreload 2

In [44]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from damage_detection_functions import *
import matplotlib.pyplot as plt
import time
import os
import copy
import json
import random
from PIL import Image
from PIL import Variable
import pprint
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

ImportError: cannot import name 'Variable' from 'PIL' (C:\Users\thoma\Anaconda3\lib\site-packages\PIL\__init__.py)

In [5]:

model_name = "resnet"

# Number of classes in the dataset
num_classes = 4

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

input_type = 'pre_and_post'

In [59]:
with open(os.path.join('C:/Users/thoma/OneDrive/Damage Detection', 'dataset_local.json')) as f:
   root_dir = json.load(f)
for image_filename in root_dir:
    print(image_filename)
    for building in root_dir[image_filename]['building_list']:
        print(building)
    break
#print(len(sorted(list(root_dir.keys()))))
#print(root_dir[image])

C:/Users/thoma/OneDrive/Damage Detection/train/images\guatemala-volcano_00000000_post_disaster.png
{'UID': '486b0813-ecd2-4b84-856c-9c0e42156953', 'damage_level': 'no-damage', 'bbox': [420.8981825090282, 165.9824503000504, 127.70380027585702, 98.49366826781642]}
{'UID': '139cf2c8-ad52-4739-82b5-bb646b215e76', 'damage_level': 'no-damage', 'bbox': [793.8653629500429, 731.7720811707778, 52.31545570319963, 43.21711558033576]}
{'UID': 'd43deb4a-529c-4df4-b666-26dd5b17e040', 'damage_level': 'no-damage', 'bbox': [976.9373337333138, 822.3286081604058, 46.559580089147175, 36.14623041059349]}
{'UID': '563b145d-732d-4eb4-8c77-380519842324', 'damage_level': 'no-damage', 'bbox': [1003.103463552428, 897.250437118042, 20.39335209665103, 41.941271667204205]}
{'UID': '56f51b26-d511-461d-bb31-747901a4ea75', 'damage_level': 'no-damage', 'bbox': [964.4984407419494, 894.1110632298545, 58.99829365411961, 105.0932754544375]}
{'UID': '7fd91a32-cf6c-43f2-b059-8c77c92a5693', 'damage_level': 'no-damage', 'bbox':

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []



    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
            else:
                train_acc_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_acc_history, val_acc_history, train_loss_history, val_loss_history

In [ ]:
#mean = np.array([0.485, 0.456, 0.406])
#std = np.array([0.229, 0.224, 0.225])

In [ ]:
#dataloaders = dataloaders_dict
#for inputs, labels in dataloaders['train']:
#    print(inputs.shape)
#    for i in range(8):
#        image = np.array(inputs[i])
#        image = np.transpose(image, (1, 2, 0))
#        image *= std
#        image += mean
#        image *= 255
#        image = image.astype("uint8")
#        plt.imshow(image)
#        plt.show()
#        print(labels[i])
#    break

In [10]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [21]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        
        num_ftrs = model_ft.fc.in_features
        
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [22]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
model_ft = nn.Sequential(*(list(model_ft.children())[:-1]))
# Print the model we just instantiated
print(model_ft)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [61]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])
def output_from_cnn(model_ft, crop):
    image_tensor = test_transforms(crop).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = image_tensor
    input = input.to(device)
    output = model_ft(input)
    return output

In [ ]:
from tqdm import tqdm
for image_filename in tqdm(root_dir):
    image = cv2.imread(image_filename)[:,:,::-1].astype("uint8")
    if 'post_disaster' in image_filename:
        path = 'C:/Users/thoma/OneDrive/Damage Detection/tensors/post'
    else:
        path = 'C:/Users/thoma/OneDrive/Damage Detection/tensors/pre'
    building_index = 0
    for building in root_dir[image_filename]['building_list']:
        bbox = building['bbox']
        x, y, width, height = bbox
        if ('pre_disaster' in image_filename) or (building['damage_level'] != 'un-classified'):
            crop = image[int(y):int(y+height), int(x):int(x+width), :]
            vector = output_from_cnn(model_ft, Image.fromarray(crop))
            vector = vector.cpu().detach().numpy()
            np.save(os.path.join(path, image_filename[54:-4] + '_tensor_' + str(building_index).zfill(5) + '.npy'), vector)
        building_index += 1
        

  1%|▋                                                                            | 49/5598 [09:05<25:39:54, 16.65s/it]

In [278]:
class DamageDetectionDataset(Dataset):
    
    def __init__(self, 
                 root_dir : dict, 
                 input_type = 'post_only', 
                 equal_distribution = True, 
                 add_damage_type = False, 
                 minimum_area = 2000, 
                 split = 'train', 
                 non_flip_transform = None
                 flip_transform = None):
        assert split in ["train", "val"]
        assert input_type in ["post_only", "pre_and_post"]
        assert add_damage_type in [True, False]
        assert equal_distribution in [True, False]
        self.root_dir = root_dir
        self.input_type = input_type
        self.add_damage_type = add_damage_type
        self.minimum_area = minimum_area
        self.split = split
        self.equal_distribution = equal_distribution
        self.non_flip_transform = non_flip_transform
        self.flip_transform = flip_transform
        self.images = [] #list of tuples
        
        if self.input_type == 'post_only':
            if self.split == 'train':
                post_images, _ = split_dataset_post_disaster(self.root_dir, percent_train= 0.8)
            else:
                _ , post_images = split_dataset_post_disaster(self.root_dir, percent_train = 0.8)

            for image_filename in post_images:
                for building in post_images[image_filename]['building_list']:
                    bbox = building['bbox']
                    if bbox[2] * bbox[3] >= self.minimum_area and building['damage_level'] != 'un-classified': #if it's not blurry
                        self.images.append(((image_filename, bbox, building['damage_level']), 0)) #0 is placeholder
                        
        elif self.input_type == 'pre_and_post':
            if self.split == 'train':
                all_images, _ = split_dataset_pre_and_post_disaster(root_dir, percent_train=0.8)
            else:
                _ , all_images = split_dataset_pre_and_post_disaster(root_dir, percent_train=0.8)
            temp_image_list = sorted(list(all_images.keys()))
            for i in range(1, len(temp_image_list), 2):
                post_image_filename = temp_image_list[i-1]
                pre_image_filename = temp_image_list[i]
                building_index = 0
                for post_building in all_images[post_image_filename]['building_list']:
                    post_bbox = post_building['bbox']
                    if post_bbox[2] * post_bbox[3] >= self.minimum_area and post_building['damage_level'] != 'un-classified':
                        pre_building = all_images[pre_image_filename]['building_list'][building_index] #corresponding pre-disaster building
                        pre_bbox = pre_building['bbox']
                        post_damage_level = post_building['damage_level']
                        post_tuple = (post_image_filename, post_bbox, post_damage_level)
                        pre_tuple = (pre_image_filename, pre_bbox)
                        self.images.append((post_tuple, pre_tuple))
                    building_index += 1
            
            
                    
        if self.equal_distribution:
            random.shuffle(self.images)
            if split == 'train': #smaller dataset of equal distribution
                per_category = 400
            else:
                per_category = 100
            dest = 0
            major = 0
            minor = 0
            none = 0 #starting with 0 buildings from each category
            temp_images = [] #temporary storage
            for buildings in self.images: #building is a tuple - if pre_and_post, then it's tuple of length 2 (pair of buildings)
                dl = buildings[0][2] #building[0] is always the post image no matter what
                if dl == 'un-classified':
                    continue
                elif dl == 'destroyed' and dest < per_category:
                    dest += 1
                    temp_images.append(buildings)
                elif dl == 'major-damage' and major < per_category:
                    major += 1
                    temp_images.append(buildings)
                elif dl == 'minor-damage' and minor < per_category:
                    minor += 1
                    temp_images.append(buildings)
                elif dl == 'no-damage' and none < per_category:
                    none += 1
                    temp_images.append(buildings)
                if dest == per_category and major == per_category and minor == per_category and none == per_category:
                    break
            self.images = temp_images #self.images now has an equal distribution of each category of building damage
            
            
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        if self.input_type == "post_only":
            (image_filename, bbox, damage_level), _ = self.images[idx]
            path = 'C:/Users/thoma/OneDrive/Damage Detection/tensors/post'
            tensor_filename = os.path.join(path, image_filename[54:-4] + '_tensor_' + building_index.zfill(5) + '.npy')
            damage_mapping = {'no-damage' : 0, 'minor-damage' : 1, 'major-damage' : 2, 'destroyed' : 3}
            sample = {'tensor': np.load(tensor_filename), 'damage_level' : damage_mapping[damage_level]}
            return sample['image'], sample['damage_level']
        
        elif self.input_type == 'pre_and_post':
            (post_image_filename, post_bbox, damage_level), (pre_image_filename, pre_bbox) = self.images[idx]
            path = 'C:/Users/thoma/OneDrive/Damage Detection/tensors/pre'
            pre_tensor_filename = os.path.join(path, pre_image_filename[54:-4] + '_tensor_' + building_index.zfill(5) + '.npy')
            path = 'C:/Users/thoma/OneDrive/Damage Detection/tensors/post'
            post_tensor_filename = os.path.join(path, post_image_filename[54:-4] + '_tensor_' + building_index.zfill(5) + '.npy')
            damage_mapping = {'no-damage' : 0, 'minor-damage' : 1, 'major-damage' : 2, 'destroyed' : 3}
            sample = {'pre_tensor' : np.load(pre_tensor_filename), 'post_tensor' = np.load(post_tensor_filename), 'damage_level' : damage_mapping[damage_level]}
            return sample['pre_tensor'], sample['post_tensor'], sample['damage_level']

In [52]:
# Data augmentation and normalization for training
# Just normalization for validation
data_non_flip_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_flip_transforms = {'train': transforms.RandomHorizontalFlip(), 'val' : None}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: DamageDetectionDataset(root_dir = root_dir, 
                                            input_type = input_type, 
                                            minimum_area = 2000, 
                                            split = x, 
                                            non_flip_transform = data_non_flip_transforms[x]
                                            flip_transform = data_flip_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

SyntaxError: invalid syntax (<ipython-input-52-e5ea3cd4beaf>, line 29)

In [270]:
# Send the model to GPU
model_ft = model_ft.to(device)
learning_rate = 0.001
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)

Params to learn:
	 conv1.weight
	 fc.weight
	 fc.bias


In [269]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [280]:
# Train and evaluate
model_ft, train_acc_hist, val_acc_hist, train_loss_hist, val_loss_hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------


KeyboardInterrupt: 

In [ ]:
epochs = range(num_epochs)
plt.plot(epochs, train_acc_hist, color = 'blue', label = 'Train')
plt.plot(epochs, val_acc_hist, color = 'orange', label = 'Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("Accuracy")
plt.show()


epochs = range(num_epochs)
plt.plot(epochs, train_loss_hist, color = 'blue', label = 'Train')
plt.plot(epochs, val_loss_hist, color = 'orange', label = 'Validation')
plt.title('Cross Entropy Loss')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Loss')
plt.legend()
plt.savefig("Cross Entropy Loss")
plt.show()


In [ ]:
model_name = ''
torch.save(model_ft.state_dict(), os.path.join("C:/Users/thoma/OneDrive/Damage Detection", model_name))